## Reinforcement Learning Fine-Tuning with GRPO for Medical Reasoning - Example Notebook
This notebook demonstrates how to fine-tune a language model using GRPO (Group Relative Policy Optimization) specifically for medical reasoning tasks. By leveraging custom reward functions and a specialized medical reasoning dataset, we transform a general-purpose language model into a domain-specific medical reasoning model.

- Installation
- Prepare data for GRPO training
- Define custom reward functions to guide the model’s learning
- Train a model using GRPO
- Test the fine-tuned model
- Save the model


## Installation

In [ ]:
# Install base requirements
!pip install -r /home/cdsw/0_session-install-dependencies/requirements.txt && pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

# Upgrade Unsloth packages
!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo

# Verify installations
import torch
import transformers
import unsloth


In [5]:
# Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms

In [6]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-24 15:09:42 __init__.py:190] Automatically detected platform cuda.


2025-03-24 15:09:45,993	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Load up `Llama 3.1 8B Instruct`, and set parameters

In [8]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0. vLLM: 0.7.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.97 GB. Also swap space = 5 GB.
WARNING 03-24 15:09:57 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 03-24 15:10:16 config.py:542] This model supports multiple tasks: {'score', 'generate', 'reward', '

[W324 15:10:20.803733361 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-24 15:10:20 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 03-24 15:10:21 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:14<00:00, 14.38s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:14<00:00, 14.38s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.05s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.05s/it]



INFO 03-24 15:10:38 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-24 15:10:38 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-24 15:10:43 worker.py:267] Memory profiling takes 4.59 seconds
INFO 03-24 15:10:43 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.50) = 7.30GiB
INFO 03-24 15:10:43 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.60GiB; the rest of the memory reserved for KV Cache is 1.08GiB.
INFO 03-24 15:10:44 executor_base.py:110] # CUDA blocks: 551, # CPU blocks: 2560
INFO 03-24 15:10:44 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 8.61x
INFO 03-24 15:10:48 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occur

Capturing CUDA graph shapes: 100%|██████████████| 19/19 [00:21<00:00,  1.15s/it]

INFO 03-24 15:11:10 model_runner.py:1562] Graph capturing finished in 22 secs, took 0.52 GiB
INFO 03-24 15:11:10 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 31.14 seconds



Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep Strategy for GRPO training
### Exploratory Data Analysis

- Do basic EDA
- Training, evaluation & testing split for reward modeling
- Consistent random state (42) for reproducibility

### Data Transformation

- Adds a structured system prompt
- Enforces a specific response format
- Separates reasoning from answer
- Prepares data for advanced medical reasoning models

### Prompt Engineering

- System prompt guides model's response structure
- Encourages step-by-step reasoning
- Separates reasoning from final answer

### Data Cleaning

- Removes unnecessary columns
- Standardizes dataset structure
- Focuses on essential information (prompt, answer, question)

### Data Prep for Reward Functions
Data prep for supporting use-case-specific training data and setting up reward functions. For data prep and all reward functions, we leverage an open-source dataset - FreedomIntelligence/medical-o1-reasoning-SFT, a dataset used to fine-tune HuatuoGPT-o1, a medical LLM designed for advanced medical reasoning.

In [ ]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd  # Needed for EDA

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def get_medical_questions(split="train", do_eda=False) -> Dataset:
    # Load the raw dataset from Hugging Face
    dataset = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', 'en')
    data = dataset[split]

    # Convert to a pandas DataFrame for easier manipulation
    df = data.to_pandas()

    # Optional EDA
    if do_eda:
        print("\n=== EDA: Previewing First 5 Rows ===")
        print(df.head())
        print("\n=== DataFrame Info ===")
        print(df.info())
        print("\n=== Null Values ===")
        print(df.isnull().sum())

    if split == "train":
        # Split into 99% train and 1% temp (for eval and test)
        train_df, temp_df = train_test_split(df, test_size=0.01, random_state=42)
        # Split 1% temp equally into eval and test
        eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

        # Convert to HF datasets
        train_dataset = Dataset.from_pandas(train_df)
        eval_dataset = Dataset.from_pandas(eval_df)
        test_dataset = Dataset.from_pandas(test_df)

        def map_fn(x):
            return {
                'prompt': [
                    {'role': 'system', 'content': SYSTEM_PROMPT},
                    {'role': 'user', 'content': x['Question']}
                ],
                'answer': x['Response'],
                'question': x['Question']
            }

        # Map and clean
        train_dataset = train_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        eval_dataset = eval_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        test_dataset = test_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])

        return train_dataset, eval_dataset, test_dataset

    else:
        # For eval/test only splits
        data = data.map(lambda x: {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': x['Question']}
            ],
            'answer': x['Response'],
            'question': x['Question']
        }).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])

        return data

In [ ]:
# Load with EDA enabled
train_dataset, eval_dataset, test_dataset = get_medical_questions(split="train", do_eda=True)

# Check one example
print("\nSample from training dataset:")
print(train_dataset[0])

## Reward Functions & Reward Calculation Process

### A. Response Extraction
- Filters and processes generated responses
- Handles edge cases:
  - Empty responses
  - Responses repeating the prompt
  - Extracting `<answer>` content

### B. Reward Calculation Steps
- **Semantic Similarity Calculation**
  - Uses `semantic_correctness()`
  - Compares generated answers with ground truth
- **Perplexity Assessment**
  - Uses `PerplexityCalculator`
  - Evaluates linguistic fluency
- **Tag Presence Evaluation**
  - Checks response structure compliance

### C. Score Combination
- Weighted combination:
  - 50% Semantic Similarity
  - 40% Perplexity Reward
  - 10% Tag Presence

### D. Normalization & Stabilization
- Handles computational edge cases
- Normalizes perplexity scores
- Clamps final rewards between -1 and 1

### E. Reward Interpretation
- -1.0: Complete failure
- 0.0: Neutral performance
- 1.0: Perfect response

## Reward Functions

The semantic correctness reward uses a cross-encoder model (cross-encoder/stsb-roberta-base) to evaluate the similarity between generated responses and reference answers. When no valid answer is extracted (i.e., an empty response), the function assigns a reward of -1.0 to indicate failure in producing an answer.

The perplexity calculation is handled by the `PerplexityCalculator` class which uses BioGPT (microsoft/biogpt) to measure the fluency and linguistic quality of responses. Note that BioGPT typically demands extra GPU resources due to its computational intensity. If a response is empty or an error occurs (e.g., a NaN loss), the function returns a default perplexity value of -1.0 to signal the issue.

The combined reward function aggregates the rewards from semantic correctness and perplexity. The rewards are weighted, with semantic similarity and perplexity contributing most significantly to form a final score. The final reward is clamped between -1 and 1, ensuring that any instance where no valid answer is found (or other issues arise) results in a -1 reward.

A Guide to Terms Used in This Notebook
======================================

| Term | Meaning in this Notebook Context |
| --- | --- |
| `prompt` | A list of messages, typically including:<br>`{'role': 'system', 'content': SYSTEM_PROMPT}`<br>`{'role': 'user', 'content': Question}` |
| `completion` | A list of messages returned by the model. Each message is typically:<br>`{'role': 'assistant', 'content': ...}`<br>The `content` contains a structured answer with `<reasoning>` and `<answer>` tags. |
| `answer` | The reference / ground truth answer from the dataset (human-written), in the same `<reasoning>...</reasoning><answer>...</answer>` format. |
| `response` | The model's generated answer **after extraction** of the `<answer>` section only (used to compare with `answer`). |

Processing Flow
---------------

1.  The model receives a medical question with instructions to provide reasoning and an answer
2.  The model generates a completion with reasoning and answer in specified tags
3.  The reward functions:
    -   Extract the answer portion from the completion
    -   Compare it to the reference answer for semantic similarity
    -   Check the perplexity of the reference answer
    -   Verify that proper tags are used
4.  The combined rewards guide the GRPO process to improve model outputs

This reward system balances accuracy (semantic similarity), fluency (perplexity), and format adherence (tag presence) to train the model to provide well-structured, accurate responses to medical reasoning questions.

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import CrossEncoder
from typing import List
import re

# ------ Device Configuration ------
main_device = "cuda" if torch.cuda.is_available() else "cpu"
reward_device = "cuda" if torch.cuda.is_available() else "cpu"

# ------ Semantic Correctness Reward ------
def semantic_correctness(responses: List[str], answers: List[str]) -> List[float]:
    """Calculate semantic similarity using cross-encoder"""
    model = CrossEncoder('cross-encoder/stsb-roberta-base', device=reward_device)
    with torch.no_grad():            
        inputs = list(zip(responses, answers))
        similarities = model.predict(inputs, show_progress_bar=False).tolist()
        # Set similarity to -1 if the response is an empty string
        similarities = [-1.0 if response == "" else similarity for response, similarity in zip(responses, similarities)]
        return similarities

# ------ Efficient Perplexity Calculation ------
class PerplexityCalculator:
    def __init__(self, model_name="microsoft/biogpt", device=reward_device):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def calculate(self, texts: List[str], batch_size=8) -> List[float]:
        perplexities = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                if not batch: continue
                
                encodings = self.tokenizer(
                    batch, 
                    return_tensors="pt", 
                    padding=True, 
                    truncation=True,
                    max_length=200
                ).to(self.device)
                
                with torch.no_grad():
                    outputs = self.model(**encodings, labels=encodings.input_ids)
                
                loss = outputs.loss
                if torch.isnan(loss):
                    raise ValueError("NaN loss encountered")
                
                batch_perplexity = torch.exp(loss).repeat(len(batch)).cpu().tolist()
                # Set perplexity to -1 if the input is an empty string
                batch_perplexity = [-1.0 if text == "" else perplex for text, perplex in zip(batch, batch_perplexity)]
                perplexities.extend(batch_perplexity)
                                
            except Exception as e:
                print(f"Error in batch {i//batch_size}: {str(e)}")
                perplexities.extend([1000.0] * len(batch))
        
        return perplexities

perplexity_calculator = PerplexityCalculator()

# ------ Tag Presence Reward ------
def tag_presence_reward(completions: List[dict]) -> List[float]:
    """Reward for presence of <reasoning> and <answer> tags"""
    rewards = []
    for completion in completions:
        content = completion[0]['content']
        has_reasoning = bool(re.search(r'<reasoning>.*?</reasoning>', content, re.DOTALL))
        has_answer = bool(re.search(r'<answer>.*?</answer>', content, re.DOTALL))
        reward = 0.5 * has_reasoning + 0.5 * has_answer
        rewards.append(reward)
    return rewards

# ------ Combined Reward Function ------
def combined_reward_func(
    prompts, completions, answer, **kwargs
) -> List[float]:
    # Extract generated responses
    responses = []
    valid_indices = []
    for idx, completion in enumerate(completions):
        try:
            generated_content = completion[0]['content'].strip()

            # Extract only the <answer> content using regex
            answer_match = re.search(r'<answer>(.*?)</answer>', generated_content, re.DOTALL)
            if answer_match:
                generated_content = answer_match.group(1).strip()
            else:
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
            
            # Skip if empty or just repeating the prompt
            user_prompt = prompts[idx][-1]['content']
            if not generated_content or generated_content == user_prompt:
                # print(f"generated_content: continue")
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
                
            responses.append(generated_content)
            # print(f"generated_content3: {generated_content}")
            valid_indices.append(idx)
        except (KeyError, IndexError):
            responses.append("")  # Append empty string to maintain index consistency
            valid_indices.append(idx)
            continue
    
    if not responses:
        return [-1.0] * len(completions)
    
    # Calculate rewards
    try:
        # Use raw answers without tag processing
        processed_answers = answer
        # print(f"processed_answers: {processed_answers[0]}")
        
        similarities = semantic_correctness(responses, [processed_answers[i] for i in valid_indices])
        perplexities = perplexity_calculator.calculate([processed_answers[i] for i in valid_indices])
        tag_rewards = tag_presence_reward([completions[i] for i in valid_indices])
    except Exception as e:
        print(f"Reward calculation error: {str(e)}")
        return [-1.0] * len(completions)
    
    # Convert to tensors with stability
    sim_scores = torch.nan_to_num(torch.tensor(similarities), nan=0.0)
    perplex_scores = torch.nan_to_num(torch.tensor(perplexities), nan=1000.0)
    tag_scores = torch.tensor(tag_rewards)
        
    # Perplexity reward calculation
    perplex_rewards = 1 / (perplex_scores / (perplex_scores.mean() + 1e-9))
    
    # Normalize with stability
    score_range = perplex_rewards.max() - perplex_rewards.min()
    if score_range < 1e-6:
        perplex_rewards_normalized = torch.ones_like(perplex_rewards) * 0.5
    else:
        perplex_rewards_normalized = (perplex_rewards - perplex_rewards.min()) / score_range
    
    # Combine scores with validation
    combined = [
        0.5 * sim.item() + 0.4 * pr.item() + 0.1 * tag.item()
        for sim, pr, tag in zip(sim_scores, perplex_rewards_normalized, tag_scores)
        if not torch.isnan(sim) and not torch.isnan(pr) and not torch.isnan(tag)
    ]
    
    # Map back to original indices
    final_rewards = [-1.0] * len(completions)
    for idx, reward in zip(valid_indices, combined):
        final_rewards[idx] = max(min(reward, 1.0), -1.0)  # Clamp between -1 and 1
    
    assert len(final_rewards) == len(completions), "Reward mapping error"
    return final_rewards

## Train the model
### Now set up GRPO Trainer and all configurations

In [14]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 2 # 2 for quicker iterations, 4 otherwise

total_steps = 200  # Changed from max steps of 1000
num_checkpoints = 4

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_generations=5,  # Decrease if out of memory
    max_prompt_length=128,  # Updated by lowering to 128 from 512 to balance longer input prompts with training time requirements
    max_completion_length=128,
    max_steps=total_steps,
    save_steps=int(total_steps // num_checkpoints),
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="grpo_outputs",
    save_strategy="steps",
)

## Let's run the trainer. 
### If you scroll, you'll see a table of rewards. The goal is to see the reward column increase. You might have to wait 150 to 200 steps for any action. 

In [16]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        combined_reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,117 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 167,772,160/8,000,000,000 (2.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / combined_reward_func
1,0.000000,-0.300000,0.000000,128.000000,0.000000,-0.300000
2,-0.000000,-0.205303,0.211749,121.900002,0.000000,-0.205303
3,0.000000,-0.300000,0.000000,124.600002,0.000010,-0.300000
4,0.000000,-0.206189,0.196091,113.900002,0.000011,-0.206189
5,0.000000,0.167601,0.410170,112.900002,0.000011,0.167601
6,0.000000,-0.209033,0.203408,113.000004,0.000013,-0.209033
7,0.000000,-0.121850,0.243961,126.100002,0.000019,-0.121850
8,0.000000,-0.205588,0.197433,123.600002,0.000012,-0.205588
9,0.000000,-0.300000,0.000000,128.000000,0.000017,-0.300000
10,0.000000,-0.300000,0.000000,119.799999,0.000046,-0.300000


TrainOutput(global_step=200, training_loss=0.0036339496986677756, metrics={'train_runtime': 11860.4504, 'train_samples_per_second': 0.034, 'train_steps_per_second': 0.017, 'total_flos': 0.0, 'train_loss': 0.0036339496986677756})

## Inference
### Now let's try the model we just trained: First, let's first try the model without any GRPO trained

In [19]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Is Aspirin good for cardio vascular function?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

Processed prompts: 100%|█| 1/1 [00:24<00:00, 24.88s/it, est. speed input: 1.85 t


In [ ]:
print(output)

"Aspirin's effect on cardiovascular function is complex and multifaceted. In small doses, aspirin is often recommended for people at risk of heart disease or those who have already had a heart attack or stroke. It acts as an antiplatelet agent, which helps prevent blood clots from forming and reducing the risk of heart attacks and strokes.\n\nHowever, it is essential to note that aspirin is not suitable for everyone, particularly those with a history of stomach ulcers or bleeding disorders. It's also not recommended for healthy individuals, as the risks may outweigh the benefits.\n\nAspirin's benefits for cardiovascular function include:\n\n1. **Antiplatelet activity**: Aspirin inhibits the production of thromboxane A2, a compound that promotes blood clotting, thus reducing the risk of blood clots forming.\n2. **Inflammation reduction**: Aspirin has anti-inflammatory properties, which can help reduce inflammation in blood vessels, a risk factor for cardiovascular disease.\n3. **Blood v

## And now with the LoRA we just trained with GRPO - we save the LoRA first:
### Now we load the LoRA and test:

In [23]:
model.save_lora("grpo_saved_lora")

In [24]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Is Aspirin good for cardio vascular function?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

Processed prompts: 100%|█| 1/1 [00:26<00:00, 26.60s/it, est. speed input: 2.56 t


In [ ]:
print(output)

"<reasoning>\nAspirin has been a topic of interest in the context of cardiovascular health for many years.\n</reasoning>\n<answer>\nAspirin may have both positive and negative effects on cardiovascular function, depending on the individual and their specific health status.\n</answer>\n\nPositive effects:\n\n1. **Anti-clotting properties**: Aspirin's antiplatelet effect can help prevent blood clots from forming, reducing the risk of heart attacks and strokes in people with a history of cardiovascular disease.\n2. **Reducing inflammation**: Aspirin has anti-inflammatory properties, which may help lower the risk of cardiovascular disease by reducing inflammation in the blood vessels.\n3. **Lowering blood pressure**: Regular, low-dose aspirin use may help lower blood pressure in people with hypertension.\n\nNegative effects:\n\n1. **Bleeding risks**: Aspirin can increase the risk of bleeding, especially in people with a history of gastrointestinal problems or taking other medications that 

## Saving your fine-tuned model

In [ ]:
model.save_pretrained_merged("model", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 7.55 out of 30.89 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  9%|████▏                                       | 3/32 [00:00<00:02, 11.16it/s]
We will save to Disk and not RAM now.
100%|███████████████████████████████████████████| 32/32 [01:58<00:00,  3.69s/it]


Unsloth: Saving tokenizer... Done.
